# Preprocess BMS data 
## refferrence
1. frog dudes masterpieces https://www.kaggle.com/c/bms-molecular-translation/discussion/231190
2. notebook fail because of output too many file take more than one hour to upload https://www.kaggle.com/product-feedback/233383

## Warnning 

this notebook is under development, just show how to proccess data parallelization and make all file into a achive file, there maybe some mistake in code 



## version 
- v1. setup from hengk's code , [drawback] output file will overcome 20 GB in about 160k pickle files
- v2 - v4.
    
    try to split into multiproccess without change output order
    
    try to split dataframe into two part to avoid overcome 20GB
- v10 
    
    try to split to 8 part to tar in case out of disk
    
- v14

finally I came up with two problem during make patchs

```
Problems: 

1. patchs pickle file will exceed 20GB

2. patch files will take more than one hour to upload , that will cause upload fail
```

```
Solutions: 

1. compress pickle file to tar.gz then kaggle will auto uncompress it after we make datasets https://www.kaggle.com/docs/datasets

2. in case of exceed 20GB during make tar file, I divide proccess into 24 parts and delete pickle files per parts,
```

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# there are too many output 
inputDir = '/kaggle/input'

def _walkdir(dirPath): 
    for dirname, _, filenames in os.walk(dirPath):
        for filename in filenames:
            print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
import random
# use for speedup reading images
import io

import cv2
import torch
import torch.nn.functional as F

import pickle
import matplotlib.pyplot as plt

import tarfile

import os.path
import shutil

In [ ]:
debug = 0

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
import subprocess
MAX_THREAD = multiprocessing.cpu_count()
MAX_THREAD

# Configuration

In [ ]:
# this mate come from henks preprocess to calculate oritation scale etc ...
data_dir = '../input/bms-molecular-translation'
csv_data_dir = '../input/bmd-mate-csv/csv'

patch_size   = 16
pixel_pad    = 3
pixel_scale  = 0.8  #1.0  #0.62=36/58 #1.0

_walkdir(csv_data_dir)

tar_file_name = 'tar-bms-moleular-translation.tar.gz'
tar = tarfile.open(tar_file_name, "w:gz")
def make_tarfile(output_filename, source_dir):
    tar.add(source_dir, arcname=os.path.basename(source_dir))

# Utils Function

In [ ]:
def write_pickle_to_file(pickle_file, x):
    with open(pickle_file, 'wb') as f:
        pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# draw -----------------------------------
def image_show(name, image, resize=1):
    H,W = image.shape[0:2]
    """
    cv2.namedWindow(name, cv2.WINDOW_GUI_NORMAL)  #WINDOW_NORMAL
    #cv2.namedWindow(name, cv2.WINDOW_GUI_EXPANDED)  #WINDOW_GUI_EXPANDED
    cv2.imshow(name, image.astype(np.uint8))
    cv2.resizeWindow(name, round(resize*W), round(resize*H))
    """
    plt.figure(figsize = (round(resize*W), round(resize*H)))
    plt.imshow(image, cmap = plt.cm.gray)
    #plt.show()

# Create patch dataset

In [ ]:
# tools to make patches ...
# see https://www.kaggle.com/yasufuminakama/inchi-resnet-lstm-with-attention-inference/data
def remove_rotate(image, orientation):
    l = orientation
    if l == 1:
        image = np.rot90(image,-1)
    if l == 2:
        image = np.rot90(image, 1)
    if l == 3:
        image = np.rot90(image, 2)
    return image


def resize_image(image, scale=1):
    if scale==1 :
        f = pixel_scale * 58/36  #1.2414 #0.80555
        b = int(round(36*0.5))

    if scale==2 :
        f = pixel_scale * 1
        b = int(round(58*0.5))

    image = image[b:-b,b:-b] #remove border
    if not np.isclose(1,f, rtol=1e-02, atol=1e-02):
        h, w = image.shape
        fw = int(round(f*w))
        fh = int(round(f*h))
        image = cv2.resize(image, dsize=(fw, fh), interpolation=cv2.INTER_AREA)
    return image

def repad_image(image, multiplier=16):
    h, w = image.shape
    fh = int(np.ceil(h/multiplier))*multiplier
    fw = int(np.ceil(w/multiplier))*multiplier
    m  = np.full((fh, fw), 255, np.uint8)
    m[0:h, 0:w] = image
    return m

def image_to_patch(image, patch_size, pixel_pad, threshold=0):
    p = pixel_pad
    h, w = image.shape

    x, y = np.meshgrid(np.arange(w // patch_size), np.arange(h // patch_size))
    yx = np.stack([y, x], 2).reshape(-1, 2)

    s = patch_size + 2*p
    m = torch.from_numpy(image).reshape(1, 1, h, w).float()
    k = F.unfold(m, kernel_size=s, stride=patch_size, padding=p)
    k = k.permute(0, 2, 1).reshape(-1, s * s)
    k = k.data.cpu().numpy().reshape(-1, s, s)
    #print(k.shape)

    sum = (1 - k[:, p:-p, p:-p]/255).reshape(len(k), -1).sum(-1)
    i = np.where(sum > threshold)
    #print(sum)
    patch = k[i]
    coord = yx[i]
    return  patch, coord

def patch_to_image(patch, coord, width, height):
    image = np.full((height,width), 255, np.uint8)
    p = pixel_pad
    patch = patch[:, p:-p, p:-p]
    num_patch = len(patch)

    for i in range(num_patch):
        y,x = coord[i]
        x = x * patch_size
        y = y * patch_size
        image[y:y+patch_size,x:x+patch_size] = patch[i]
        cv2.rectangle(image, (x, y), (x + patch_size, y + patch_size), 128, 1)
    return image

#<todo>
# np compression is very slow!!!
# https://stackoverflow.com/questions/39035983/compress-zip-numpy-arrays-in-memory
def compress_array(k):
    compressed_k = io.BytesIO()
    np.savez_compressed(compressed_k, k)
    return compressed_k

def uncompress_array(compressed_k):
    compressed_k.seek(0)
    k  = np.load(compressed_k,allow_pickle=True)['arr_0']
    return k

#<todo> add token to mark 4 corner of image
#############################################################################################
def make_chessbord_image(w, h, patch_size=16):
    m = np.zeros((h, w), np.float32)
    s = patch_size
    u = 1
    for y in range(0, h, s):
        v = u
        for x in range(0, w, s):
            m[y:y + s, x:x + s] = v
            v *= -1
        u *= -1
    m = ((0.5 * m + 0.5) * 255).astype(np.uint8)
    return m


# Divide data into two part

In [ ]:
mode = 'train'
# outout will be too large so we have to divide into two part
PARTNUM = 24
#_PART_ = 0
#dump_dir = './bms-moleular-translation-part' + str(_PART_)
dump_dir = './bms-moleular-translation'

dump_dir_mutaion = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']


if mode == 'train':
    df = pd.read_csv(csv_data_dir+'/df_train_image_meta.csv')
    folder = 'train_patch16_s%0.3f'%(pixel_scale)

if mode == 'test':
    df = pd.read_csv(csv_data_dir+'/df_test_image_meta.csv')
    folder = 'test_patch16_s%0.3f'%(pixel_scale)

#df = df.head(4000)
# calculate the chunk size as an integer
part_size = round(df.shape[0]/PARTNUM)
# split dateframe 
part_chunks = [df.iloc[df.index[i:i + part_size]] for i in range(0, df.shape[0], part_size)]

# just for part1



In [ ]:
# multiprocess data iterrows https://stackoverflow.com/questions/40357434/pandas-df-iterrows-parallelization
        
def _parafunc(df):
    num_patch = []
    for i, d in df.iterrows():
        image_id = d.image_id
        if i%1000==0: print(i, image_id)
        scale = d.scale
        orientation = d.orientation


        image_file = data_dir + '/%s/%s/%s/%s/%s.png' % (mode, image_id[0], image_id[1], image_id[2], image_id)

        image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        if mode=='test':
            image = remove_rotate(image, orientation)

        image = resize_image(image, scale)
        image = repad_image(image, patch_size)  # remove border and repad        

        #k, yx = image_to_patch(image, patch_size, pixel_pad, threshold=pixel_scale*4)
        k, yx = image_to_patch(image, patch_size, pixel_pad, threshold=4)

        """
            if debug and i == 0:  #debug
            for y, x in yx:
                x = x * patch_size
                y = y * patch_size
                cv2.rectangle(image, (x, y), (x + patch_size, y + patch_size), 128, 1)

            image_show('image', image, resize=1)
            #cv2.waitKey(0)
        """
        #-------------------------------------------
        h,w = image.shape
        yx  = yx.astype(np.int32)
        k   = compress_array(k.astype(np.uint8))
        write_pickle_to_file(dump_dir + '/%s/%s/%s/%s/%s.pickle' % (
            folder, image_id[0], image_id[1], image_id[2], image_id),
            {'patch':k, 'coord': yx, 'width': w,'height': h}
        )
        num_patch.append(len(yx))
    return num_patch
    
def run_make_patch_data():
    
    num_patch = []
        
    print(mode, "total data length" , len(df))
    
    for _PART_ in range(PARTNUM):
        
        _PARTITION_DF_ = part_chunks[_PART_]
        print(mode, "part", _PART_ , "data length" , len(_PARTITION_DF_))
        
        for f in dump_dir_mutaion:
            for g in dump_dir_mutaion:
                for h in dump_dir_mutaion:
                    os.makedirs(dump_dir+'/%s/%s/%s/%s'%(folder,f,g,h),exist_ok=True)


        # divide data frame by multiprocessing thread
        chunk_size = int(_PARTITION_DF_.shape[0]/MAX_THREAD)
        _shape = _PARTITION_DF_.shape[0]
        chunks = [_PARTITION_DF_.iloc[_PARTITION_DF_.index[i:i + chunk_size] - part_size *_PART_ ] for i in range(0, _shape, chunk_size)]

        # create our pool with `num_processes` processes
        pool = multiprocessing.Pool(processes=MAX_THREAD)

        # apply our function to each chunk in the list        
        result = pool.map(_parafunc, chunks)
        num_patch += [item for sublist in result for item in sublist]
        pool.close()
        pool.join()

        """

        num_patch2 = []
        for i,d in _PARTITION_DF_.iterrows():
            if i%1000==0: print(i, d.image_id)
            image_id = d.image_id
            scale = d.scale
            orientation = d.orientation

            image_file = data_dir + '/%s/%s/%s/%s/%s.png' % (mode, image_id[0], image_id[1], image_id[2], image_id)
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

            if mode=='test':
                image = remove_rotate(image, orientation)

            image = resize_image(image, scale)
            image = repad_image(image, patch_size)  # remove border and repad        

            #k, yx = image_to_patch(image, patch_size, pixel_pad, threshold=pixel_scale*4)
            k, yx = image_to_patch(image, patch_size, pixel_pad, threshold=4)

            h,w = image.shape
            yx  = yx.astype(np.int32)
            k   = compress_array(k.astype(np.uint8))

            write_pickle_to_file(dump_dir + '/%s/%s/%s/%s/%s.pickle' % (
                folder, d.image_id[0], d.image_id[1], d.image_id[2], d.image_id),
                {'patch':k, 'coord': yx, 'width': w,'height': h}
            )
            num_patch2.append(len(yx))
        """
        
        make_tarfile(tar_file_name, dump_dir)
        shutil.rmtree(dump_dir)
        
    #print(num_patch == num_patch2)
    
    df_patch = pd.DataFrame({
        'image_id': df.image_id.values,
        'num_patch': num_patch
    })
    _cvsfile_name = './df_train_patch_s%0.3f.csv'%(pixel_scale)
    df_patch.to_csv(_cvsfile_name, index=False)
    
    make_tarfile(tar_file_name, _cvsfile_name)
    os.remove(_cvsfile_name)
    
    tar.close()
    exit(0)

In [ ]:
run_make_patch_data()